In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
import torchvision.datasets as dset
import torch.optim as optim
from torch.autograd import Variable
import torchvision.utils as vutils
import torch.nn.functional as F


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
image_size = 64
num_epochs = 20
batch_size = 128

In [3]:
transform = transforms.Compose([transforms.Resize((image_size, image_size)), 
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), 
                                                     (0.5, 0.5, 0.5)),])
dataset = dset.ImageFolder("/home/tyler/data/image/celebs/", 
                               transform)
# dataset = dset.CIFAR10(root = '/home/tyler/data/image', 
#                        download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size = batch_size, 
                                         shuffle = True, 
                                         num_workers = 2)

In [4]:
## weight initalization for network
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

### Generator

Produces an image

Q: why the convtranspose2d numbers? A: they are the inverse of discriminator

Q: Why first convtranspose2d in channels is 100? This is just a hyper-parameter. You can choose how many channels you want to randomly generate for the input to your generator.

In [5]:
class Generator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Generator, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(100, d*8, 4, 1, 0, bias=False)
        self.deconv1_bn = nn.BatchNorm2d(d*8)
        self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1, bias=False)
        self.deconv2_bn = nn.BatchNorm2d(d*4)
        self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1, bias=False)
        self.deconv3_bn = nn.BatchNorm2d(d*2)
        self.deconv4 = nn.ConvTranspose2d(d*2, d, 4, 2, 1, bias=False)
        self.deconv4_bn = nn.BatchNorm2d(d)
        self.deconv5 = nn.ConvTranspose2d(d, 3, 4, 2, 1, bias=False)

    # forward method
    def forward(self, input):
        # x = F.relu(self.deconv1(input))
        x = F.relu(self.deconv1_bn(self.deconv1(input)))
        x = F.relu(self.deconv2_bn(self.deconv2(x)))
        x = F.relu(self.deconv3_bn(self.deconv3(x)))
        x = F.relu(self.deconv4_bn(self.deconv4(x)))
        x = F.tanh(self.deconv5(x))

        return x

## Discriminator

Just a normal conv net to tell if an image is fake or not.

In [6]:
class Discriminator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, d, 4, 2, 1, bias=False)
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1, bias=False)
        self.conv2_bn = nn.BatchNorm2d(d*2)
        self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1, bias=False)
        self.conv3_bn = nn.BatchNorm2d(d*4)
        self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1, bias=False)
        self.conv4_bn = nn.BatchNorm2d(d*8)
        self.conv5 = nn.Conv2d(d*8, 1, 4, 1, 0, bias=False)

    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.conv1(input), 0.2)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = F.sigmoid(self.conv5(x))

        return x.view(-1)

In [7]:
netG = Generator(128).to(device)
netG.apply(weights_init)
netD = Discriminator(128).to(device)
netD.apply(weights_init)

Discriminator(
  (conv1): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv2_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv3_bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv4_bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
)

In [8]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.00005, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

In [10]:
total_step = len(dataloader)
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        
        ## train discriminator
        
        netD.zero_grad()
        
        ## calculate error using real image 
        real, _ = data
        input = Variable(real).to(device)
        # target is 1 b/c real image
        target = Variable(torch.ones(input.size()[0])).to(device)
        output = netD(input)
        real_score = output
        errD_real = criterion(output, target)
        
        ## calculate error using fake image
        ## first generate an image using generator then discriminate
        ## this is 100 channels, 1x1 random noise that the generate will use
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)).to(device)
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0])).to(device)
        output = netD(fake.detach())
        fake_score = output
        errD_fake = criterion(output, target)
        
        
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()
        
        
        ## train generator
        
        
        ## we want the generator to learn to create realistic images
        ## and thus to produce a 1 from the discriminator
        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0])).to(device)
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, errD.item(), errG.item(), 
                    real_score.mean().item(), fake_score.mean().item()))
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

Epoch [0/20], Step [200/1583], d_loss: 1.0883, g_loss: 0.7075, D(x): 0.40, D(G(z)): 0.05
Epoch [0/20], Step [400/1583], d_loss: 0.5744, g_loss: 1.8120, D(x): 0.64, D(G(z)): 0.09
Epoch [0/20], Step [600/1583], d_loss: 0.6264, g_loss: 2.5629, D(x): 0.84, D(G(z)): 0.34
Epoch [0/20], Step [800/1583], d_loss: 0.6424, g_loss: 4.1927, D(x): 0.90, D(G(z)): 0.39
Epoch [0/20], Step [1000/1583], d_loss: 0.5288, g_loss: 2.3670, D(x): 0.79, D(G(z)): 0.23
Epoch [0/20], Step [1200/1583], d_loss: 0.4631, g_loss: 2.8292, D(x): 0.89, D(G(z)): 0.27
Epoch [0/20], Step [1400/1583], d_loss: 0.5043, g_loss: 2.5886, D(x): 0.83, D(G(z)): 0.25
Epoch [1/20], Step [200/1583], d_loss: 0.5426, g_loss: 1.9748, D(x): 0.68, D(G(z)): 0.12
Epoch [1/20], Step [400/1583], d_loss: 0.7102, g_loss: 1.0610, D(x): 0.64, D(G(z)): 0.19
Epoch [1/20], Step [600/1583], d_loss: 0.5103, g_loss: 2.6900, D(x): 0.79, D(G(z)): 0.21
Epoch [1/20], Step [800/1583], d_loss: 0.9821, g_loss: 0.5204, D(x): 0.43, D(G(z)): 0.06
Epoch [1/20], Step

Epoch [13/20], Step [400/1583], d_loss: 0.1943, g_loss: 3.6659, D(x): 0.95, D(G(z)): 0.13
Epoch [13/20], Step [600/1583], d_loss: 0.4784, g_loss: 5.5407, D(x): 0.99, D(G(z)): 0.34
Epoch [13/20], Step [800/1583], d_loss: 0.1688, g_loss: 3.6962, D(x): 0.90, D(G(z)): 0.06
Epoch [13/20], Step [1000/1583], d_loss: 0.2143, g_loss: 2.6228, D(x): 0.87, D(G(z)): 0.06
Epoch [13/20], Step [1200/1583], d_loss: 0.2012, g_loss: 3.4621, D(x): 0.87, D(G(z)): 0.05
Epoch [13/20], Step [1400/1583], d_loss: 0.2281, g_loss: 3.8010, D(x): 0.95, D(G(z)): 0.14
Epoch [14/20], Step [200/1583], d_loss: 0.2455, g_loss: 3.1290, D(x): 0.87, D(G(z)): 0.09
Epoch [14/20], Step [400/1583], d_loss: 0.1628, g_loss: 2.8261, D(x): 0.90, D(G(z)): 0.05
Epoch [14/20], Step [600/1583], d_loss: 0.2571, g_loss: 2.7045, D(x): 0.86, D(G(z)): 0.09
Epoch [14/20], Step [800/1583], d_loss: 0.3071, g_loss: 2.7076, D(x): 0.93, D(G(z)): 0.19
Epoch [14/20], Step [1000/1583], d_loss: 0.2297, g_loss: 4.5993, D(x): 0.95, D(G(z)): 0.15
Epoch 